In [ ]:
import torch
import torchvision.transforms as T

from data_loader import get_dataloader
from models.dual_model import DualModel
from loss import ContrastiveLoss
from one_epoch import train_one

In [ ]:
INTRA = True
YEAR = '2015'
SENSOR = 'CrossMatch'
DATASET_PATH = '/home/hmb1604/projects/FinPAD/data'

BATCH_SIZE = 32
NUM_WORKERS = 4

EMB_DIM = 1024

LR = 1e-3
WEIGHT_DECAY = 1e-4
NUM_EPOCHS = 100

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = {
    'Train': T.Compose([
        T.Resize((224, 224)),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.RandomAffine(
            degrees=(-20, 20),          # Rotation
            translate=(0.2, 0.2),       # Horizontal/vertical shift
            shear=(-20, 20),            # Shear
            scale=(0.8, 1.2),           # Zoom
            interpolation=T.InterpolationMode.NEAREST,
            fill=0
        ),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),

    'Test': T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

In [ ]:
train_loader, val_loader, train_label_map = get_dataloader(intra=INTRA, year=YEAR, sensor=SENSOR, dataset_path=DATASET_PATH, train=True, binary_class=True, transform=transform, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
test_loader, test_label_map = get_dataloader(intra=INTRA, year=YEAR, sensor=SENSOR, dataset_path=DATASET_PATH, train=False, binary_class=True, transform=transform, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

Loading Test Ecoflex: 100%|██████████| 270/270 [00:00<00:00, 977102.74it/s]


In [ ]:
model = DualModel(num_classes=EMB_DIM).to(device=device)
model = torch.nn.DataParallel(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/hmb1604/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [01:45<00:00, 5.23MB/s] 


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/hmb1604/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:17<00:00, 5.88MB/s]


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = ContrastiveLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

In [ ]:
print('Begin training...')
for epoch in range(NUM_EPOCHS):
    train_one()

